## Installing modules

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


## Accessing dataset

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', as_supervised=True, with_info=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [3]:
print(mnist_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='C:\\Users\\shrit\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)


In [4]:
num_validataion_samples = 0.1*mnist_info.splits['train'].num_examples
num_validataion_samples = tf.cast(num_validataion_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [6]:
scaled_VandT_mnist = mnist_train.map(scale)
scaled_test_mnist = mnist_test.map(scale)

In [7]:
BUFFER = 10000
shuffeled_TandV = scaled_VandT_mnist.shuffle(BUFFER)


In [8]:
validation_data = shuffeled_TandV.take(num_validataion_samples)
train_data = shuffeled_TandV.skip(num_validataion_samples)

In [9]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validataion_samples)

# Test data
scaled_test_mnist.shuffle(BATCH_SIZE)
test_data = scaled_test_mnist.batch(num_test_samples)



In [10]:
validation_inputs, validation_targets = next(iter(validation_data))

## Model

In [11]:
input_size = 784
hidden_layer_size = 200
output_size = 10

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1,28,28)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')

])

d:\Python_works\ML\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Choosing optimizer and loss function

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [13]:
NUM_EPOCHS=5
model.fit(train_data, epochs =NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)


Epoch 1/5
540/540 - 14s - 26ms/step - accuracy: 0.9167 - loss: 0.2730 - val_accuracy: 0.9620 - val_loss: 0.1275
Epoch 2/5
540/540 - 5s - 9ms/step - accuracy: 0.9666 - loss: 0.1095 - val_accuracy: 0.9697 - val_loss: 0.1055
Epoch 3/5
540/540 - 5s - 9ms/step - accuracy: 0.9755 - loss: 0.0791 - val_accuracy: 0.9785 - val_loss: 0.0748
Epoch 4/5
540/540 - 5s - 10ms/step - accuracy: 0.9816 - loss: 0.0602 - val_accuracy: 0.9823 - val_loss: 0.0618
Epoch 5/5
540/540 - 5s - 9ms/step - accuracy: 0.9842 - loss: 0.0516 - val_accuracy: 0.9782 - val_loss: 0.0724


## Testing

In [14]:
test_accuracy, test_loss = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 860ms/step - accuracy: 0.9732 - loss: 0.0968
